# Sequential Rigid Body Object Stacking

A greedy, physics-based next best object pose planning algorithm 

In [1]:
import pybullet as p
import os 
import time
import pybullet_data
import threading
import numpy as np
import scipy
import trimesh
%matplotlib inline
from Rocks.TowerCenterOfMass import convex_decomposition_center_of_mass_calculation

lmap = lambda fn,it: list(map(fn,it))
final_data_path = './Rocks/SimulatorRockObjDefinitions'
data_dir = './Rocks/SingleRockFiles_Processed'
rock_dirs = os.listdir(data_dir)
MESH_SCALE = .03

In [2]:
class NextBestPoseSearch: 
    
    def __init__(self, rock_tower, non_tower_rocks): 
        # Rocks that we can consider candiates to place on top 
        # of the existing tower during the pose planning phase 
        self.non_tower_rocks = non_tower_rocks
        
        # The tower that contains all already stacked rocks 
        self.rock_tower = rock_tower

    def run(): 
        
        z_value = rock_tower.get_com()
        for rock in self.non_tower_rocks: 
            rock_tower


class RockTower: 
    '''
    Abstraction to encapsulate information about the tower of rocks. 
    
    [Rock] - rocks: 
        A list of rock objects 
    '''
    
    def __init__(self, defaultPosition):
        self.defaultPosition
        #Tower always starts empty 
        self.rocks = []
        self.posehash = {}
        
    def get_com(self): 
        '''
        Get the center of mass of the tower 
        '''
        if len(self.rocks) == 0:
            return self.defaultPosition
        coms = [rock.get_com() for rock in self.rocks]
        masses = [rock.get_mass() for rock in self.rocks]
        total_mass = sum(masses)
        weighted_coms = []
        for com, mass in zip(coms, masses):
            mass_p = mass / total_mass
            weighted_coms.append(lmap(lambda coord: coord * mass_p, com))
        weighted_coms = np.array(weighted_coms).reshape((-1,3))
        return np.mean(weighted_coms, axis=0)
            
    def set_gravity(self, gravityEnabled): 
        '''
        Set the gravity to on / off for all objects in the tower 
        '''
        for rock in self.rocks: 
            rock.toggle_gravity(gravityEnabled)
        
    def revert_to_hash_state(self): 
        '''
        Revert to the last stable tower state where all object have gravity disabled 
        and exist at a fixed position and orientation 
        '''
        for rock in self.rocks:
            pose = self.posehash[rock.sim_id]
            rock.toggle_gravity(False)
            rock.set_pos_ori(pos=pose['pos'], 
                             ori=pose['ori'])
        
    def hash_state(self): 
        '''
        Prior to re-enabling gravity, we call this method to record the stable state 
        values for each rock in the tower. If when computing the cost, these values change at 
        all, we reset the tower before the next cost computation 
        '''
        for rock in self.rocks: 
            pos, ori = p.getBasePositionAndOrientation(rock.sim_id)
            self.posehash[rock.sim_id] = { "pos": pos, "ori": ori }
        
    def add_rock(rock): 
        pass
        
    def reset_existing_tower(): 
        pass
    
        
    def get_polygon_of_support(self):
        pass
        

class Rock: 
    '''
    Abstraction representing our physics simulator rock object
    '''
    
    def __init__(self, rock_path, hulls, com, volume): 
        self.rock_path = rock_path
        self.hulls = hulls 
        self.com = com
        self.com_inv = lmap(lambda e: -e, com) 
        self.mass = volume
        self.gravityEnabled = True 
        self.initialPosition = [0,0,0]
        self.initialPosition[0] += self.com_inv[0]
        self.initialPosition[1] += self.com_inv[1]
        self.initialPosition[2] += self.com_inv[2]
        self.sim_id = None #will remain None until rock is spawned in simulation 
        
    def toggle_gravity(self, gravityOn): 
        assert(self.sim_id)
        self.gravityEnabled = gravityOn
        if self.gravityEnabled: 
            p.changeDynamics(self.sim_id, -1, mass=self.mass)
        else: 
            p.changeDynamics(self.sim_id, -1, mass=0)
        
    def get_gravity_enabled(self): 
        assert(self.sim_id)
        return self.gravityEnabled
        
    def get_sim_id(self):
        assert(self.sim_id)
        return self.sim_id
    
    def get_mass(): 
        return self.mass 
    
    def get_com(self): 
        # The position of the base is the object center of mass 
        assert(self.sim_id)
        cur_pos, cur_ori = p.getBasePositionAndOrientation(self.sim_id)
        return cur_pos 
        
    def get_pos_ori(self):
        assert(self.sim_id)
        return p.getBasePositionAndOrientation(self.sim_id)
    
    def set_pos_ori(self, pos=None, ori=None):
        '''
        Set the position and or orientation of the rock in the simulation 
        '''
        assert(self.sim_id)
        cur_pos, cur_ori = p.getBasePositionAndOrientation(self.sim_id)
        do_change = pos is not None or ori is not None 
        
        # is position or orientation is unspecified, simply keep previous values 
        if pos is None: 
            pos = cur_pos
        if ori is None: 
            ori = cur_ori
            
        # only re-render if necessary 
        if do_change: 
            p.resetBasePositionAndOrientation(self.sim_id, pos, ori)
    
    def spawn(self, hasGravity=True): 
        '''
        Builds a rock from a .obj definition file (visual / collision components)
        Spawns the rock at a given location in the physics simulator (relative to center of mass)

        Returns: 
            String: The id of the created object 
        '''
        
        mass = self.mass if hasGravity else 0 

        #Create visual component 
        visualId = p.createVisualShape(p.GEOM_MESH, 
                                       fileName=self.rock_path, 
                                       meshScale=[MESH_SCALE for i in range(3)])
        #Create colliosion component 
        collisionId = p.createCollisionShape(p.GEOM_MESH, 
                                             fileName=self.rock_path, 
                                             meshScale=[MESH_SCALE for i in range(3)])
        
        #Link visual/collison components to create dynamic rigid body in environment 
        rockId = p.createMultiBody(baseCollisionShapeIndex=collisionId, 
                                   baseVisualShapeIndex=visualId, 
                                   baseMass=mass, 
                                   basePosition=self.initialPosition, 
                                   baseInertialFramePosition=self.com)
        
        #add dynamic properties to the rigid body 
        p.changeDynamics(rockId, 
                         -1, #every multi-body consists of only a single base link, referenced as -1
                         lateralFriction=.25, #contact friction
                         restitution=.25 #bouncyness 
                        )
        
        #update sim_id from None to the spawned object id 
        self.sim_id = rockId
        

In [3]:

def construct_rocks(logging=True):
    '''
    Iterate over a collection of rock files. Each file needs to be parsed 
    to compute center of mass and volume approximations which are stored
    for later use during the physics engine setup / stacking algorithm 
    '''
    rocks = []

    for rdir in rock_dirs: 
        obj_files = [f for f in os.listdir(f"{data_dir}/{rdir}") if '.obj' in f]
        obj_paths = [f'{data_dir}/{rdir}/{f}' for f in obj_files]
        assert(len(obj_files) > 0)
        o_counter = 0
        new_file = ''
        hulls = []
        total_volume = 0
        for objp in obj_paths: 
            new_file += f'o object{o_counter}\n'
            o_counter += 1
            with open(objp) as objfile: 
                file_part = objfile.read()
                file_lines = [fl for fl in file_part.split('\n') if fl[0] == 'v'] 
                hull_coords = [lmap(lambda a : MESH_SCALE * float(a), fl.split(' ')[1:])
                               for fl in file_lines]
                hull = scipy.spatial.ConvexHull(hull_coords)
                total_volume += hull.volume 
                hulls.append(hull)
                new_file += file_part + '\n\n'
        final_rock_path = f'{final_data_path}/{rdir}.obj'
        com = convex_decomposition_center_of_mass_calculation(hulls)

        rock = Rock(final_rock_path, hulls, com, total_volume)
        rocks.append(rock)

        with open(final_rock_path, 'w') as write_rock_file: 
            write_rock_file.write(new_file)

        if logging: 
            print(f'wrote: {final_rock_path}')
    
    return rocks
        

In [ ]:

'''
Utility function to spawn a number of rocks into the environment so we can 
use them for the physics based stacking algorithm 
'''


def spawn_operable_rocks(rocks, initialPosition=[-20,30,0]): 
    '''
    Spawn a collection of static rock objects over which our physics based algorithm will
    operate. Ensure that none of the rocks intersect with each other and that none of the 
    rocks are near the stacking tower. These spawned rocks will have NO gravity. 
    '''
    e = .05
    last_bbox = [[initialPosition[0] - e, initialPosition[1] - e, initialPosition[2] - e],
                 [initialPosition[0] + e, initialPosition[1] + e, initialPosition[2] + e]]
        
    for rock in rocks:  
                
        # Spawn the rock in the simulation, compuate the initial axis aligned bounding box 
        rock.spawn(hasGravity=True)
        new_bbox = p.getAABB(rock.sim_id)
        
        # Compute a transformation that places the new bounding box the the right 
        # of the existing bounding box by computing a change of basis transformation 
        # between the 3d vectors to_pt and from_pt 
        to_pt =   [last_bbox[1][0],
                  (last_bbox[0][1] + last_bbox[1][1]) / 2,
                  (last_bbox[0][2] + last_bbox[1][2]) / 2]
        
        from_pt = [new_bbox[0][0],
                  (new_bbox[0][1] + new_bbox[1][1]) / 2,
                  (new_bbox[0][2] + new_bbox[1][2]) / 2]
        
        # Compute the coordinate shift that must be applied to the object 
        dx = to_pt[0] - from_pt[0]
        dy = to_pt[1] - from_pt[1]
        dz = to_pt[2] - from_pt[2]
        
        cur_pos, cur_ori = rock.get_pos_ori()
        
        new_pos = [
            cur_pos[0] + dx,
            cur_pos[1] + dy,
            cur_pos[2] + dz
        ]
        
        # Set the new rock position 
        rock.set_pos_ori(pos=new_pos)
        
        # update the old bbox 
        last_bbox = p.getAABB(rock.sim_id)
        

In [ ]:
#Setup the physics engine. 
physicsClient = p.connect(p.GUI) #or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath()) #optionally
p.setGravity(0,0,0) #world has no universal gravity, we apply gravity to individual objects as need be 

#Create a platform upon which we will perform stacking 
planeId = p.loadURDF("plane.urdf", basePosition=[0,0,-10])

wrote: ./Rocks/SimulatorRockObjDefinitions/rock5.obj
wrote: ./Rocks/SimulatorRockObjDefinitions/rock2.obj
wrote: ./Rocks/SimulatorRockObjDefinitions/rock3.obj
wrote: ./Rocks/SimulatorRockObjDefinitions/rock4.obj
wrote: ./Rocks/SimulatorRockObjDefinitions/rock16.obj
wrote: ./Rocks/SimulatorRockObjDefinitions/rock29.obj
wrote: ./Rocks/SimulatorRockObjDefinitions/rock11.obj
wrote: ./Rocks/SimulatorRockObjDefinitions/rock27.obj
wrote: ./Rocks/SimulatorRockObjDefinitions/rock18.obj
wrote: ./Rocks/SimulatorRockObjDefinitions/rock20.obj
wrote: ./Rocks/SimulatorRockObjDefinitions/rock21.obj
wrote: ./Rocks/SimulatorRockObjDefinitions/rock26.obj
wrote: ./Rocks/SimulatorRockObjDefinitions/rock19.obj
wrote: ./Rocks/SimulatorRockObjDefinitions/rock10.obj
wrote: ./Rocks/SimulatorRockObjDefinitions/rock17.obj
wrote: ./Rocks/SimulatorRockObjDefinitions/rock28.obj
wrote: ./Rocks/SimulatorRockObjDefinitions/rock30.obj
wrote: ./Rocks/SimulatorRockObjDefinitions/rock1.obj
wrote: ./Rocks/SimulatorRockObjDe

In [ ]:
use_n_rocks = 5
rocks = construct_rocks()[:use_n_rocks]
spawn_operable_rocks(rocks)

In [ ]:
i = 0

def axis_aligned_rays(vec3, dv): 
    return [
        [vec3[0] + dv, vec3[1], vec3[2]],
        [vec3[0], vec3[1] + dv, vec3[2]],
        [vec3[0], vec3[1], vec3[2] + dv]
    ]

def add_axis_aligned_rays_at_point(vec3, dv=5):
    lines = axis_aligned_rays(vec3)
    p.addUserDebugLine(vec3, lines[0], [1,0,0])
    p.addUserDebugLine(vec3, lines[1], [0,1,0])
    p.addUserDebugLine(vec3, lines[2], [0,0,1])
    
while 1: 
    p.stepSimulation()
    p.setGravity(0,0,-10)
    i += 1
    if i % 10 == 0: 
        for rock in rocks: 
            com = rock.get_com()
            add_axis_aligned_rays_at_point(com)
            
#         print("doing the thing")
#         for r in rocks: 
#             try: 
#                 print(p.getNumJoints(r.sim_id))
#             except Exception as e: 
#                 pass
#         pos = [0,0,0]
#         quat = p.getQuaternionFromEuler([np.random.uniform(0,np.pi*2) for i in range(3)])
#         rocks[0].set_pos_ori(pos=pos,
#                              ori=quat)

    time.sleep(1/240)